In Sentence Transformers, a "CrossEncoder" is a type of model architecture that is designed to encode and compare pairs of sentences simultaneously. Unlike the "Siamese" architecture, where the two sentences are encoded separately and then their representations are compared, the CrossEncoder processes both sentences together in a single forward pass.

The CrossEncoder architecture is commonly used for tasks that involve comparing pairs of sentences, such as sentence similarity, semantic textual similarity, paraphrase identification, and natural language inference. It's especially useful when you want to calculate a single similarity score or label for the entire sentence pair.

Here's a high-level overview of how a CrossEncoder works:

1. **Input Encoding:** The two sentences in the pair are tokenized and encoded together as a single input. They are processed through a pre-trained transformer-based model, such as BERT, RoBERTa, or other variants, which produces a fixed-length vector representation for the entire sentence pair.

2. **Similarity Calculation:** The encoded sentence pair representation is then passed through one or more fully connected layers or other transformation functions to calculate a similarity score. This score indicates how similar the two sentences are in terms of their semantic meaning, with higher scores indicating higher similarity.

3. **Training:** During training, CrossEncoder models are often fine-tuned using supervised learning on tasks that require sentence pair comparisons. The model is trained with labeled data, where each sentence pair is associated with a similarity score or a binary label (e.g., paraphrase/non-paraphrase).

The CrossEncoder architecture is particularly useful when you need a single model that can handle multiple types of similarity-related tasks, as it can provide efficient and accurate sentence pair comparisons.

**What is the NLI task in NLP?**

Natural Language Inference (NLI) is the task of determining whether the given “hypothesis” logically follows from the “premise”. In layman's terms, you need to understand whether the hypothesis is true, while the premise is your only knowledge about the subject.

#### MultiNLI
 The Multi-Genre Natural Language Inference (MultiNLI) corpus is a crowd-sourced collection of 433k sentence pairs annotated with textual entailment information. The corpus is modeled on the SNLI corpus, but differs in that covers a range of genres of spoken and written text, and supports a distinctive cross-genre generalization evaluation. The corpus served as the basis for the shared task of the RepEval 2017 Workshop at EMNLP in Copenhagen. 

**Examples**

Premise 	                                            Label 	    Hypothesis

The Old One always comforted Ca'daan, except today. 	neutral 	Ca'daan knew the Old One very well.

Your gift is appreciated by each and every student 
who will benefit from your generosity.                  neutral 	Hundreds of students will benefit from your generosity.

yes now you know if if everybody like in August when 
everybody's on vacation or something we can dress 
a little more casual or 	                            contradiction 	August is a black out month for vacations in the company.


At the other end of Pennsylvania Avenue, people began 
to line up for a White House tour. 	                    entailment 	People formed a line at the end of Pennsylvania Avenue.

#### Fine Tuning SBERT

In [ ]:
%pip install sentence-transformers   
%pip install torch torchvision torchaudio

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import torch
torch.__version__

#### Goo Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### My personal traning dataset

In [ ]:
!cp /content/my_new_ds_train_eval_test.zip /content/drive/My\ Drive/Colab\ Notebooks/ML\ Project\ 2/my_new_ds_train_eval_test.zip

#### Download NLI Traning Dataset

In [ ]:
from torch.utils.data import DataLoader
import math

from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import gzip
import csv
import pandas as pd

In [2]:
import pandas as pd

In [21]:
df = pd.read_csv('data.csv')

In [22]:
df

,text,hypothesis,label
0,Linux is an open-source operating system known...,"Windows,a widely used operating system develop...",contradiction
1,"Linux,with its command-line interface,provides...","Windows,unlike Linux,requires a graphical user...",contradiction
2,"Linux distributions,such as Ubuntu,Fedora,and ...","Windows,being a commercial operating system,co...",contradiction
3,"Windows,despite being a widely used operating ...","Linux,with its frequent software updates and r...",entailment
4,"Linux,due to its lightweight and efficient des...","Windows,as a more resource-intensive operating...",entailment
...,...,...,...
259,Machine learning enables computers to learn fr...,Artificial Intelligence has transformed variou...,contradiction
260,Sorting algorithms are crucial for arranging d...,Data structures are fundamental for efficient ...,entailment
261,Agile development emphasizes iterative and cus...,Software development methodologies like Agile ...,entailment
262,"React.js,a popular front-end framework,offers ...",Web development involves front-end and back-en...,entailment


In [ ]:
# Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

#As dataset, we use SNLI + MultiNLI
#Check if dataset exsist. If not, download and extract  it
nli_dataset_path = 'datasets/AllNLI.tsv.gz'

if not os.path.exists(nli_dataset_path):
    util.http_get('https://sbert.net/datasets/AllNLI.tsv.gz', nli_dataset_path)


# Read the AllNLI.tsv.gz file and create the training dataset
logger.info("Read AllNLI train dataset")

In [ ]:
label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}

train_samples = []
dev_samples = []

In [ ]:
with gzip.open(nli_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        label_id = label2int[row['label']]
        if row['split'] == 'train':
            train_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=label_id))
        else:
            dev_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=label_id))

In [ ]:
with gzip.open(nli_dataset_path, 'rt', encoding='utf8') as fIn:
    df_x = pd.read_csv(fIn, sep='\t', names=['sentence1', 'sentence2','label_id'], quoting=csv.QUOTE_NONE)

In [ ]:
df_x.head(1)

In [ ]:
df_x.head(1)['sentence1']

In [ ]:
df_x.head(1)['sentence2']

In [ ]:
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_allnli-' + \
    datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
# Define our CrossEncoder model. We use distilroberta-base as basis and setup it up to predict 3 labels
model = CrossEncoder('distilroberta-base', num_labels=len(label2int))

In [ ]:
# We wrap train_samples, which is a list ot InputExample, in a pytorch DataLoader
train_dataloader = DataLoader(
    train_samples, shuffle=True, batch_size=train_batch_size)

# During training, we use CESoftmaxAccuracyEvaluator to measure the accuracy on the dev set.
evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(
    dev_samples, name='AllNLI-dev')


# 10% of train data for warm-up
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
logger.info("Warmup-steps: {}".format(warmup_steps))

**But it takes too time like 20 hours for traning**

In [ ]:
# Train the model
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=10000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

#### Use already fine-tuned Cross-Encoder model

In [ ]:
# pretrained model
model_hugg = CrossEncoder('cross-encoder/nli-distilroberta-base', max_length=512)

In [ ]:
texts = [
    ("good public policy should make society healthier happier  safer and more productive", "scoial sciencists should be able to help us understand the world better"),
]

In [ ]:
texts[0]

In [ ]:
scores_hugg = model_hugg.predict(texts)

In [ ]:
# convert computed liast of list equal scores_hugg to simple labels
label_mapping = ["contradiction", "entailment", "neutral"]

# original scores
print(scores_hugg)

In [ ]:
# for easy reading: convert list to singular label
labels = [label_mapping[score_max] for score_max in scores_hugg.argmax(axis=1)]
print(labels)

### Train Cross Encoder

In [ ]:
train_btach_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_continue_training-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

### Train Cross Encoder Model on Custom DataSet

In [ ]:
train_batch_size = 16
num_epochs = 3
model_save_path = 'output/my_training_ds-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
# cross encoder model
my_model = CrossEncoder("distilroberta-base", num_labels=len(label2int), max_length=512)

# distilroberta-base is a smaller version of RoBERTa-base. It has 6 layers, 768 hidden size, 12 attention heads and about 82M parameters.

# distilroberta-base is a smaller version of RoBERTa-base. It has 6 layers, 768 hidden size, 12 attention heads and about 82M parameters.

In [1]:
label2int = {
    'contradiction': 0,
    'entailment': 1,
    'neutral': 2
}

rows = [
    {
        'sentence1': 'I like to eat apples.',
        'sentence2': 'Apples are my favorite fruit.',
        'label': 1
    },
    {
        'sentence1': 'I like to eat apples.',
        'sentence2': 'Apples are my favorite fruit.',
        'label': 1
    }
]

In [ ]:
train_samples_my = []
for i in range(len(rows)):
    train_samples_my.append(InputExample(texts=[train_samples[i]['sentence1'], train_samples[i]['sentence2']], label=train_samples[i]['label']))

In [ ]:
rows[0][0]

In [ ]:
test_samples = []

In [ ]:
train_dataloader_my = DataLoader(train_samples_my, shuffle=True,batch_size=train_batch_size)

In [ ]:
evaluator_my = CESoftmaxAccuracyEvaluator.from_input_examples(test_samples, name='my-test')

In [ ]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

# warmup steps is a parameter that controls the learning rate during the first few epochs of training.
# During the warm-up steps, the learning rate is linearly increased from 0 to the specified learning rate.
# After the warm-up phase, the learning rate is decreasing again during training.
# This is a common technique in transfer learning, especially for fine-tuning BERT models.


logger.info("Warmup-steps: {}".format(warmup_steps))

In [ ]:
# Train the model
my_model.fit(train_dataloader=train_dataloader,
             evaluator=evaluator,
             epochs=1,
             evaluation_steps=2000,
             warmup_steps=warmup_steps,
             save_best_model=True,
             output_path=model_save_path)
# evaluation_steps means how often the model should be evaluated on the dev set.
# save_best_model=True means that only the best model is saved on disk (correctly classified most dev samples)
